In [349]:
import os
import pandas as pd

In [350]:
#create filepath
school_filepath = os.path.join("raw_data","schools_complete.csv")
student_filepath = os.path.join("raw_data","students_complete.csv")

In [351]:
#read csv to data-frames
school_df = pd.read_csv(school_filepath)
student_df = pd.read_csv(student_filepath)

In [352]:
school_df.head()

,School ID,name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [353]:
student_df.head()

,Student ID,name,gender,grade,school,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [354]:
#use groupby on student_df to get the avg reading/math score for each school
stu_df1 = student_df.groupby("school")["reading_score","math_score"].mean()
stu_df1

,reading_score,math_score
school,,
Bailey High School,81.033963,77.048432
Cabrera High School,83.975780,83.061895
Figueroa High School,81.158020,76.711767
Ford High School,80.746258,77.102592
Griffin High School,83.816757,83.351499
Hernandez High School,80.934412,77.289752
Holden High School,83.814988,83.803279
Huang High School,81.182722,76.629414
Johnson High School,80.966394,77.072464


In [355]:
#use column filter on student_df to find the % of students passing math & reading(assuume 70 is pass)
math_pass_df = student_df.loc[student_df["math_score"] > 60, :]

reading_pass_df = student_df.loc[student_df["reading_score"] > 70, :]

In [356]:
#use groupby on each of the df to get the total passing students per school
math_pass_count = pd.DataFrame(math_pass_df.groupby("school")["name"].count())
#rename "name" column 
math_pass_count = math_pass_count.rename(columns = {"name":"total math pass"})

reading_pass_count = pd.DataFrame(reading_pass_df.groupby("school")["name"].count())
#rename "name" column 
reading_pass_count = reading_pass_count.rename(columns = {"name":"total reading pass"})


In [357]:
#Use df.join() to join these two df to stu_df1
stu_df2 = stu_df1.join(math_pass_count, how = "outer")
stu_df3 = stu_df2.join(reading_pass_count, how = "outer") 
stu_df3.head()

,reading_score,math_score,total math pass,total reading pass
school,,,,
Bailey High School,81.033963,77.048432,4351,3946
Cabrera High School,83.975780,83.061895,1858,1744
Figueroa High School,81.158020,76.711767,2549,2313
Ford High School,80.746258,77.102592,2389,2123
Griffin High School,83.816757,83.351499,1468,1371


In [358]:
#make the index of stu_df4(school) into a column
stu_df3.reset_index(level=0, inplace=True)
stu_df3.head()

,school,reading_score,math_score,total math pass,total reading pass
0,Bailey High School,81.033963,77.048432,4351,3946
1,Cabrera High School,83.975780,83.061895,1858,1744
2,Figueroa High School,81.158020,76.711767,2549,2313
3,Ford High School,80.746258,77.102592,2389,2123
4,Griffin High School,83.816757,83.351499,1468,1371


In [359]:
#rename "name" column in school_df to "school" to facilitate merging
school_df = school_df.rename(columns = {"name":"school"})
school_df.head()

,School ID,school,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [360]:
#merge school_df & stu_df4
merge_df = pd.merge(school_df, stu_df3, on = "school", how = "outer")
merge_df.head()

,School ID,school,type,size,budget,reading_score,math_score,total math pass,total reading pass
0,0,Huang High School,District,2917,1910635,81.182722,76.629414,2533,2299
1,1,Figueroa High School,District,2949,1884411,81.158020,76.711767,2549,2313
2,2,Shelton High School,Charter,1761,1056600,83.725724,83.359455,1761,1631
3,3,Hernandez High School,District,4635,3022020,80.934412,77.289752,4007,3624
4,4,Griffin High School,Charter,1468,917500,83.816757,83.351499,1468,1371


In [361]:
#calculate the pass % per school for reading & math
merge_df["% passing math"] = (merge_df["total math pass"]/merge_df["size"])*100
merge_df["% passing reading"] = (merge_df["total reading pass"]/merge_df["size"])*100
merge_df.head()

,School ID,school,type,size,budget,reading_score,math_score,total math pass,total reading pass,% passing math,% passing reading
0,0,Huang High School,District,2917,1910635,81.182722,76.629414,2533,2299,86.835790,78.813850
1,1,Figueroa High School,District,2949,1884411,81.158020,76.711767,2549,2313,86.436080,78.433367
2,2,Shelton High School,Charter,1761,1056600,83.725724,83.359455,1761,1631,100.000000,92.617831
3,3,Hernandez High School,District,4635,3022020,80.934412,77.289752,4007,3624,86.450917,78.187702
4,4,Griffin High School,Charter,1468,917500,83.816757,83.351499,1468,1371,100.000000,93.392371


In [362]:
#create a df with only information about district schools
district_df = merge_df.loc[merge_df["type"] == "District", : ]

In [363]:
#get data from district_df 
total_district_schools = district_df["school"].count()
total_district_students = district_df["size"].sum()
total_district_budget = district_df["budget"].sum()
avg_math_score = district_df["math_score"].mean()
avg_reading_score = district_df["reading_score"].mean()
prcnt_passing_math = district_df["% passing math"].mean()
prcnt_passing_reading = district_df["% passing reading"].mean()
overall_passing_rate = (prcnt_passing_math + prcnt_passing_reading)/2

In [364]:
#create a data frame from the district data
district_summary = pd.DataFrame({"Overall Passing Rate" : [overall_passing_rate],
                                 "% Passing Reading" : [prcnt_passing_reading],
                                 "% Passing Math": [prcnt_passing_math],
                                 "Average Reading Score" : [avg_reading_score],
                                 "Average Math Score" : [avg_math_score],
                                 "Total Budget" :[total_district_budget],
                                 "Total Students" : [total_district_students],
                                 "Total Schools" : [total_district_schools],
                                })
district_summary

,% Passing Math,% Passing Reading,Average Math Score,Average Reading Score,Overall Passing Rate,Total Budget,Total Schools,Total Students
0,86.790742,78.324559,76.956733,80.966636,82.557651,17347923,7,26976


In [365]:
#arrange items in the correct order
district_summary.loc[:,["Total Schools","Total Students","Total Budget","Average Math Score","Average Reading Score",\
                       "% Passing Math","% Passing Reading","Overall Passing Rate"]]

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,7,26976,17347923,76.956733,80.966636,86.790742,78.324559,82.557651
